In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("datasets/bus.csv")

grouped = df.groupby('route_full_name')['on_time_percent'].mean().reset_index()

colors = []
for percentage in grouped['on_time_percent']:
    if percentage >= 0.8:
        colors.append('green')
    elif percentage >= 0.65:
        colors.append('yellow')
    else:
        colors.append('red')

plt.figure(figsize=(12, 30))
plt.barh(grouped['route_full_name'], grouped['on_time_percent'], color=colors, height = .8)

plt.title('Bus On-Time Performance by Neighborhood')
plt.xlabel('On-Time Percentage')
plt.ylabel('Neighborhood')

plt.show()
